In [10]:

!pip install python-chess

In [11]:
import chess.pgn
import numpy as np
import chess
import pandas as pd

In [12]:
import numpy

In [ ]:
import chess

def board_to_matrix_play(board):
    piece_map = board.piece_map()
    matrix = np.zeros((8, 8, 1), dtype=np.int8)
    for square, piece in piece_map.items():
        matrix[square // 8, square % 8, 0] = piece.piece_type * (1 if piece.color == chess.WHITE else -1)
    return matrix

def find_best_move(board, model):
    legal_moves = list(board.legal_moves)
    best_move = None
    best_score = -np.inf
    for move in legal_moves:
        board.push(move)
        pred = model.predict(board_to_matrix_play(board).reshape(1, 8, 8, 1), verbose=0)
        score = np.max(pred)
        if score > best_score:
            best_score = score
            best_move = move
        board.pop()
    return best_move

def play_against_model(model):
    board = chess.Board()
    while not board.is_game_over():
        print(board)
        user_move = input("Your move (or type 'abort' to quit): ")
        if user_move.lower() == 'abort':
            print("Game aborted.")
            break
        try:
            move = board.parse_san(user_move)
            if move in board.legal_moves:
                board.push(move)
            else:
                print("Illegal move. Try again.")
                continue
        except Exception:
            print("Invalid move. Try again.")
            continue
        if board.is_game_over():
            print("Game over:", board.result())
            break
        model_move = find_best_move(board, model)
        if model_move is None:
            print("Model cannot find a move. Game over.")
            break
        print(f"Model move: {board.san(model_move)}")
        board.push(model_move)
    print("Final board:")
    print(board)

# Load and play
from tensorflow.keras.models import load_model
model = load_model('/content/chess_cnn_model (1).keras')
play_against_model(model)

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
Your move (or type 'abort' to quit): e4
Model move: f5
r n b q k b n r
p p p p p . p p
. . . . . . . .
. . . . . p . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
Your move (or type 'abort' to quit): Nf5
Invalid move. Try again.
r n b q k b n r
p p p p p . p p
. . . . . . . .
. . . . . p . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
Your move (or type 'abort' to quit): Nf6
Invalid move. Try again.
r n b q k b n r
p p p p p . p p
. . . . . . . .
. . . . . p . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
Your move (or type 'abort' to quit): d4
Model move: e5
r n b q k b n r
p p p p . . p p
. . . . . . . .
. . . . p p . .
. . . P P . . .
. . . . . . . .
P P P . . P P P
R N B Q K B N R
Your move (or type 'abort' to quit): BF5
Invalid move. Try again.
r n b q k b n r
p p p p . . p p
. . . . . . . .
. . 